# Computational Methods in Economics

## Problem Set - Root Finding: Suggested Solutions

In [1]:
# Author: Alex Schmitt (schmitt@ifo.de), Christina Littlejohn (littlejohn@ifo.de)

import datetime
print('Last update: ' + str(datetime.datetime.today()))

Last update: 2018-01-11 15:07:03.649826


### Preliminaries

#### Import Modules

In [2]:
import numpy as np
import scipy.optimize

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

import numpy as np
import scipy.optimize
import scipy.linalg

## Question 1 (N)

Write a function **mybisect(f, a, b)** in Python that implements the pseudo-code for the bisection method from the lecture. Then, test it on the function 
\begin{equation*}
    f(x) = \sin(4 (x - 1/4)) + x + x^{20} - 1,
\end{equation*}
i.e. find a root of this function. Compare your result to what SciPy's in-built function returns. 

*Hint*: most modern programming languages have some type of **while**-loop, which will prove useful here. Moreover, in Python/NumPy, consider using the **abs()** and **np.sign()** functions.  

In [3]:
# function to use bisection on
def fun(x):
    return np.sin(4 * (x - 0.25)) + x + x**20 - 1


def mybisect(fun, a, b):
    """
    Implements the bisection method
    """
    # choose tolerance level
    tol = 1e-10
    # initialize d 
    d = 1
    # while-loop: iterate until d sufficiently small
    while abs(d) > tol:
        # find intermediate value between a and b
        x = (a + b)/2
        # evaluate function
        d = fun(x)
        # find new end points for interval [a,b]
        if np.sign(d) == np.sign(fun(a)):
            a = x
        elif np.sign(d) == np.sign(fun(b)):
            b = x
    
    return x

print(mybisect(fun,0,2))        
print(scipy.optimize.bisect(fun,0,2))

0.408293504267931
0.4082935042806639


## Question 2 (N)

Solve the example about the Cournot Duopoly in M&F, p. 35 ff., in Python using Newton's method, and compare your result to M&F.

In [3]:
def cournot(x):
    """
    Implements a system of equations in two unknowns, here the Cournot Duopoly model
    f(x) = [(q1 + q2)^(-1/eta) - (1/eta)*[(q1 + q2)^(-1/eta - 1)]*q_i - c_i*q_i]
    """
    c = [0.6, 0.8]
    eta = 1.6
    e = -1/eta
    
    return np.array(( (x[0]+x[1])**e + e*((x[0]+x[1])**(e-1))*x[0] - c[0]*x[0],
                    (x[0]+x[1])**e + e*((x[0]+x[1])**(e-1))*x[1] - c[1]*x[1]))

def fun_J(x):
    """
    Implements the Jacobian system of equation in two unknowns above
    """
    c = [0.6, 0.8]
    eta = 1.6
    e = -1/eta
    
    f_00 = (e*(x[0]+x[1])**(e-2))*(e*x[0] + x[0] + 2*x[1]) - c[0] 
    f_01 = (e*(x[0]+x[1])**(e-2))*(e*x[0] + x[1])
    f_10 = (e*(x[0]+x[1])**(e-2))*(e*x[1] + x[0])
    f_11 = (e*(x[0]+x[1])**(e-2))*(e*x[1] + 2*x[0] + x[1]) - c[1]
    
    return np.array([[f_00, f_01], [f_10, f_11]])

In [4]:
def my_newton_mult(fun, fun_J, x,  tol = 1e-8, tol2 = 1e-6):
    """
    Implements Newton's method for a vector-valued function
    """    
    eps = 1
    it = 0
    while eps > tol:
        it += 1
        f, J = fun(x), fun_J(x)
        x_new = x - np.linalg.inv(J) @ f
        eps = np.linalg.norm(x - x_new)
        x = x_new
    
    print("Number of iterations = {}".format(it) )
    
    if np.linalg.norm(fun(x)) < tol2: 
        return x
    else:
        print("No solution found!")
    
    return x
        
x_init = [0.2, 0.2]
x = my_newton_mult(cournot, fun_J, x_init)
print(x)

Number of iterations = 6
[ 0.8395676   0.68879643]


## Question 3 (N)

Modify the pseudo-code for Newton's method to include backstepping, as outlined in the lecture. Then, include backstepping to the Python function **my_newton**. 

Pseudo-code for Newton's method with backstepping:

(i) Specify tolerance levels $tol1$ and $tol2$ and choose a starting guess $x^{(0)}$.

(ii) Compute the suggested step as 

\begin{equation}
 s^{(k)} = - J(x^{(k)})^{-1} f(x^{(k)})
\end{equation}

(iii) If $\left| \left|\ f(x^{(k)} + s^{(k)}) \right| \right|$ < $\left| \left|\ f(x^{(k)}) \right| \right|$, set $x^{(k+1)} = x^{(k)} + s^{(k)}$ and go to (vi).

(iv) Otherwise, if $\left| \left|\ f\left(x^{(k)} + \frac{s^{(k)}}{2}\right) \right| \right|$ < $\left| \left|\ f(x^{(k)} + s^{(k)}) \right| \right|$, set $s^{(k)} = \frac{s^{(k)}}{2}$ and go back to (iii).

(v) Otherwise, set $x^{(k+1)} = x^{(k)} + s^{(k)}$.

(vi) Check the stopping rule: if $\left| \left|\ x^{(k+1)} - x^{(k)}) \right| \right| < tol1$, stop. If not, go back to (ii).

(vii) If $\left| \left|\ f(x^{(k)}) \right| \right| < tol2$, report $x^{(k+1)}$ as the solution. Otherwise, report failure. 

In [7]:
def my_newton_bs(fun, fun_d, x,  tol1 = 1e-8,  tol2 = 1e-8, maxit = 100):
    """
    Implements Newton's method for a vector-valued function with backstepping
    """    
    dist = 1
    it = 0
    
    while dist > tol1 and it < maxit:
        it += 1
        f, J = fun(x), fun_d(x)
        ## step (ii): compute suggested step
        s = - np.linalg.inv(J) @ f
        
        ## start backstepping routine
        while np.linalg.norm( fun(x + s) ) > np.linalg.norm( fun(x) ): ## step (iii)
            if np.linalg.norm( fun(x + 0.5 * s) ) < np.linalg.norm( fun(x + s) ): # step (iv)
                s = 0.5 * s
            else:   # step (v)
                break ## terminate inner while loop 
        
        x_new = x + s
        dist = np.linalg.norm(x - x_new)
        x = x_new
    
    print("Number of iterations = {}".format(it) )
    
    if np.linalg.norm( fun(x) ) < tol2:
        return x
    else:
        print('No solution found!')


Compare the example from the lecture:

In [8]:
def fun_vv(x):
    """
    Implements a system of equation in two unknowns, here f(x) = [x2**2 - 1; sin(x1) - x2]
    """
    return np.array( (x[1]**2 - 1 , np.sin(x[0]) - x[1] ) )


def fun_J(x):
    """
    Implements the Jacobian system of equation in two unknowns above
    """
    f_00 = 0
    f_01 = 2 * x[1]
    f_10 = np.cos(x[0])
    f_11 = -1
    
    return np.array([[f_00, f_01], [f_10, f_11]])  

        
x_init = [1.5,0.9]
x = my_newton_bs(fun_vv, fun_J, x_init)
print(x)

Number of iterations = 24
[ 1.57079633  1.        ]


## Question 4 (N)

Write a function **mysecant(f, x0)** in Python that implements the pseudo-code for the secant method from the lecture. Again, test it on the function $f$ and compare the result to question 1.

In [9]:
def g_secant(fun, f_old, x, x_old):
    """
    Implements the update rule for the secant method
    """
    f = fun(x)
    fd = (f - f_old) / (x - x_old)

    return x - f * fd**(-1), f


def my_secant(fun, x, tol1 = 1e-8,  tol2 = 1e-8, maxit = 100):
    """
    Implements the secant method for a univariate scalar function
    """        
    dist = 1
    it = 0
    
    x_old = 0.9 * x
    f_old = fun(x_old)

    while dist > tol1 and it < maxit:
        it += 1
        x_new, f = g_secant(fun, f_old, x, x_old)
        dist = abs(x - x_new)
        
        ## store "old" function and x value for next iteration
        f_old = f
        x_old = x
        
        x = x_new
        print(x_new)

    print("Number of iterations = {}".format(it) )
    
    if abs(fun(x)) < tol2: 
        return x
    else:
        print("No solution found!")

In [10]:
%timeit -r1 -n1 my_secant(fun, 0.01)

0.56032103275
0.439946290312
0.398938029656
0.408647798005
0.408297121241
0.408293502842
0.408293504279
Number of iterations = 7
1 loop, best of 1: 1.38 ms per loop


## Question 5 (N)

Solve question 3.7 in M&F, p. 55.

The problem of agent $i$ is given by

\begin{equation}
    \max U_i(x) = \sum_{j = 1}^2 a_{ij} (v_{ij} + 1)^{-1} x_{ij}^{v_{ij} + 1} 
\end{equation}

s.t. $p_1 x_{i1} + p_2 x_{i2} = p_1 e_{i1} + p_2 e_{i2}$. 


Taking first-conditions gives 

\begin{equation}
    \frac{\partial U_i(x)}{\partial x_{ij}} = U_{ij}(x) = a_{ij} x_{ij}^{v_{ij}} = \lambda_i p_j
\end{equation},

where $\lambda_i$ is the Lagrange multiplier corresponding to agent $i$.

Hence, we have three first order conditions, for $i \in \{1, 2, 3\}$:

\begin{equation}
     \frac{ a_{i1} x_{i1}^{v_{i1}} }{p_1} - \frac{ a_{i2} x_{i2}^{v_{i2}} }{p_2} = 0
\end{equation},

Moreover, we have three budget constraints:

\begin{equation}
    p_1 x_{i1} + p_2 x_{i2} - p_1 e_{i1} - p_2 e_{i2} = 0
\end{equation}



In [11]:
## read parameters
A = np.array([ [2.0, 1.5],
               [1.5, 2.0],
               [1.5, 2.0]  ])

V = np.array([ [-2.0, -0.5],
               [-1.5, -0.5],
               [-0.5, -1.5] ])

E = np.array([ [2.0, 3.0],
               [1.0, 2.0],
               [4.0, 0.0]  ])

In [12]:
## define residual function

def S(z, A, V, E):
    
    X = np.array([ [np.exp(z[0]), np.exp(z[1])],
               [np.exp(z[2]), np.exp(z[3])],
               [np.exp(z[4]), np.exp( z[5])]  ])
    p1 = np.exp( z[6])
    
    S = np.zeros(7)
    
    ### f.o.c.'s
    for i in range(3):
        S[i] = A[i, 0] * X[i, 0]**V[i, 0] / p1 - A[i, 1] * X[i, 1]**V[i, 1] / (1 - p1)
    
    ## budget constraints
    for i in range(2):
        S[3 + i] = p1 * X[i, 0] + (1 - p1) * X[i, 1] - (p1 * E[i, 0] + (1 - p1) * E[i, 1])
    
    S[5] = sum(X[:, 1]) - sum(E[:, 1])
    
    ## resource constraint good 1
    S[6] = sum(X[:, 0]) - sum(E[:, 0])
    
    
    return S


z0 = np.array([0, 0, 0, 0, 0, 0, -0.5])

res = scipy.optimize.root(S, z0, method = 'broyden2', args = (A, V, E))
print(res)
print( np.exp( res.x ) )

     fun: array([ -1.54201222e-06,   1.57843350e-07,   2.97911288e-06,
        -3.95860673e-07,  -3.38857131e-06,  -3.33916917e-06,
         2.00401352e-06])
 message: 'A solution was found at the specified tolerance.'
     nit: 59
  status: 1
 success: True
       x: array([ 1.06485863,  1.01623987,  0.86227679,  0.49436689,  0.54871421,
       -0.51458278, -1.56775534])
[ 2.90042891  2.76278677  2.36854725  1.63945995  1.73102584  0.59774993
  0.2085127 ]


## Question 6 (A)

(a) Show that the update rule for $A^{(k+1)}$ used in Broyden's method,

\begin{equation}
 A^{(k+1)} = A^{(k)} + \frac{ \left( \mathbf{f}(\mathbf{x}^{(k+1)}) - \mathbf{f}(\mathbf{x}^{(k)}) - A^{(k)} \mathbf{p}^{(k)} \right) (\mathbf{p}^{(k)})^T}{(\mathbf{p}^{(k)})^T \mathbf{p}^{(k)}}
\end{equation}

satisfies the secant condition,

\begin{equation}
 A^{(k+1)} \mathbf{p}^{(k)} = \mathbf{f}(\mathbf{x}^{(k+1)}) - \mathbf{f}(\mathbf{x}^{(k)}).
\end{equation}

 and

\begin{equation}
 A^{(k+1)} \mathbf{q} = A^{(k)} \mathbf{q}\ \ \text{for}\ \ \mathbf{q}^{T} \mathbf{p}^{(k)} = 0
\end{equation}.

(b) To prepare question (c), Show that for any vector $\mathbf{p} \in \mathbb{R}^n$, we have 

\begin{equation}
    \left| \left| \frac{\mathbf{p}\ \mathbf{p}^T}{\mathbf{p}^T \mathbf{p} } \right| \right| = 1
\end{equation}


(c) Using the result from question (b), show that

\begin{equation}
 A^{(k+1)} \in \arg \min_{A :\ A \mathbf{p}^{(k)} = \mathbf{f}(\mathbf{x}^{(k+1)}) - \mathbf{f}(\mathbf{x}^{(k)})} ||\ A - A^{(k)} ||
\end{equation}

Hint: Use the update rule in (a) to rewrite the distance $||\ A^{(k+1)}  - A^{(k)} ||$.